## Sobrecarga de claves de índices secundarios globales
Puede crear 20 índices secundarios globales para una tabla de DynamoDB en el momento en que se escribió esta página. Sin embargo, en ocasiones, es posible que su aplicación necesite admitir varios patrones de acceso y superar el límite actual de índices secundarios globales por tabla. El patrón de diseño de sobrecarga de claves de índice secundario global se habilita designando y reutilizando un nombre de atributo (encabezado de columna) en diferentes tipos de elementos y almacenando un valor en ese atributo en función del contexto del tipo de elemento. Cuando se crea un índice secundario global en ese atributo, se está indexando para múltiples patrones de acceso, cada uno para un tipo de elemento diferente, y sólo se ha utilizado 1 índice secundario global. Por ejemplo, una tabla de empleados. Un empleado puede contener elementos de tipo metadatos (para los detalles del empleado), empleado-título (todos los títulos de trabajo que ha tenido el empleado) o empleado-ubicación (todos los edificios de oficinas y ubicaciones donde ha trabajado el empleado).

Los patrones de acceso necesarios para este escenario son

* Consulta de todos los empleados de un estado
* Consulta de todos los empleados con un título actual específico
* Consulta de todos los empleados que han tenido alguna vez un cargo específico
* Consulta de empleados por nombre

In [ ]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from spdynamodb import DynamoTable
import random
import json
import time
from decimal import Decimal

In [ ]:
dt = DynamoTable()
try:
    dt.select_table('employees')
    print(dt)
except:
    dt.create_table(
        table_name='employees',
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S',
        read_capacity=5,
        write_capacity=5,
        tags=[{'Key': 'workshop-design-patterns', 'Value': 'targeted-for-cleanup'}]
)

In [ ]:
dt.create_global_secondary_index(
    att_name="GSI1-PK",
    att_type="S",
    sort_index="GSI1-SK",
    sort_type="S",
    index_name="GSI1",
    read_capacity=5,
    write_capacity=5
)

In [ ]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    time.sleep(30)
print("Global secondary index created.")

In [ ]:
columns = ['EmployeeID', 'Name', 'JobTitle', 'Department', 'Location', 'State', 'HireDate', 'TerminationDate', 'NewJobTitle', 'PromotionDate', 'ManagerID']
df = pd.read_csv("data/employees.csv", names=columns, header=None)
df['PK'] = df.apply(lambda x: 'e#' + str(x['EmployeeID']), axis=1)
df.head()

| Attribute Name (Type) | Special Attribute? | Attribute Use Case | Sample Attribute Value |
|-----------------------|--------------------|---------------------|------------------------|
| PK (STRING)           | Partition Key      | Employee ID         | e#129                  |
| SK (STRING)           | Sort key           | Derived value       | root, state#MIGSI_1_PK |
| GSI_1_PK (STRING)     | GSI_1 partition key| Derived value       | root, state            |
| GSI_1_SK (STRING)     | GSI_1 sort key     | Employee name       | Christine Milsted      |


In [ ]:
from python_code.load_employees import import_csv

import_csv(
    fileName="data/employees.csv",
    dynamodb_table=dt.table
)